![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)
## Deploy Azure SQL Edge to an existing device via IoT hub
This notebook will walk you through process of deploying Azure SQL Edge to an existing device of an IoT hub:
1. Deploy Azure SQL Edge module to the device with optional dacpac
1. If a dacpac is selected, a storage account will be created to host the dacpac file
1. Enable connecting to the Azure SQL Edge instance on the device

### Dependencies
- Azure CLI. For more information, see [Azure CLI Installation](https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest).

<span style="color:red"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### Check dependencies

In [ ]:
import pandas,sys,os,json,html,getpass,time,ntpath,uuid
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')

def run_command(command:str, displayCommand:str = "", returnObject:bool = False):
    print("Executing: " + (displayCommand if displayCommand != "" else command))
    if returnObject:
        output = os.popen(command).read()
        print(f'Command successfully executed')
        return json.loads(''.join(output))
    else:
        !{command}
        if _exit_code != 0:
            sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n')
        else:
            print(f'Command successfully executed')

run_command(command='az --version')

### Install Azure IoT extension for Azure CLI

In [ ]:
run_command('az extension add --name azure-cli-iot-ext')

### Required information

In [ ]:
azure_subscription_id = os.environ["AZDATA_NB_VAR_ASDE_SUBSCRIPTIONID"]
azure_resource_group = os.environ["AZDATA_NB_VAR_ASDE_RESOURCEGROUP"]
sa_password = os.environ["AZDATA_NB_VAR_SA_PASSWORD"]
dacpac_path = os.environ["AZDATA_NB_VAR_ASDE_DACPAC_PATH"]
sql_port = os.environ["AZDATA_NB_VAR_ASDE_SQL_PORT"]
iot_hub_name = os.environ["AZDATA_NB_VAR_ASDE_HUBNAME"]
iot_device_id = os.environ["AZDATA_NB_VAR_ASDE_DEVICE_ID"]
ip_address = os.environ["AZDATA_NB_VAR_ASDE_DEVICE_IP_ADDRESS"]
print(f'Subscription: {azure_subscription_id}')
print(f'Resource group: {azure_resource_group}')
print(f'IoT hub name: {iot_hub_name}')
print(f'Device ID: {iot_device_id}')
print(f'Device IP address: {ip_address}')
print(f'Azure SQL Edge instance port: {sql_port}')
print(f'Azure SQL Edge instance sa password: ******')
print(f'Dacpac path: {dacpac_path}')

### Default settings

In [ ]:
suffix = time.strftime("%y%m%d%H%M%S", time.localtime())
azure_storage_account = f'sa{suffix}'
storage_account_container = 'sqldatabasedacpac'
sql_lcid = '1033'
sql_collation = 'SQL_Latin1_General_CP1_CI_AS'

### Login to Azure

In [ ]:
run_command('az login')

### Set active Azure subscription

In [ ]:
if azure_subscription_id != "":
    run_command(f'az account set --subscription {azure_subscription_id}')
else:
    print('Using the default Azure subscription', {azure_subscription_id})
run_command(f'az account show')

### Make sure the IoT hub and device both exist

In [ ]:
hub_list = run_command(f'az iot hub list --resource-group {azure_resource_group}', returnObject=True)
hub_list = [hub for hub in hub_list if hub['name'] == iot_hub_name]
if len(hub_list) == 0:
    sys.exit(f'IoT hub \"{iot_hub_name}\" does not exist.')

device_list = run_command(f'az iot hub device-identity list --edge-enabled true --hub-name {iot_hub_name} --resource-group {azure_resource_group}', returnObject=True)
device_list = [device for device in device_list if device['deviceId'] == iot_device_id]
if len(device_list) == 0:
    sys.exit(f'Edge device \"{iot_device_id}\" does not exist.')

### Create storage account and storage account container, then upload the dacpac

In [ ]:
if dacpac_path == "":
    print(f'Dacpac zip file not provided')
    blob_sas = ''
else:
    dacpac_name = ntpath.basename(dacpac_path)
    storage_accounts = run_command(f'az storage account list --resource-group {azure_resource_group} --subscription {azure_subscription_id}', returnObject=True)
    storage_accounts = [storage_account for storage_account in storage_accounts if storage_account['name'] == azure_storage_account]
    if len(storage_accounts) == 0:
        run_command(f'az storage account create -n {azure_storage_account} -g {azure_resource_group} --sku Standard_LRS --kind Storage')
    else:
        print(f'storage account \"{azure_storage_account}\" already exists.')

    storage_account_key = run_command(f'az storage account keys list --account-name {azure_storage_account} --resource-group {azure_resource_group}', returnObject=True)[0]['value']
    container_exists = run_command(f'az storage container exists --name {storage_account_container} --account-key {storage_account_key} --account-name {azure_storage_account} --auth-mode key --output json', returnObject=True)['exists']
    if container_exists:
        print(f'storage account container \"{storage_account_container}\" already exists.')
    else:
        run_command(f'az storage container create --name {storage_account_container} --account-key {storage_account_key} --account-name {azure_storage_account} --auth-mode key')

    blob_exists = run_command(f'az storage blob exists --container-name {storage_account_container} --name \"{dacpac_name}\" --account-key {storage_account_key} --account-name {azure_storage_account} --auth-mode key', returnObject=True)['exists']
    if blob_exists:
        print(f'blob \"{dacpac_name}\" already exists.')
    else:
        run_command(f'az storage blob upload --account-name {azure_storage_account} --container-name {storage_account_container} --name {dacpac_name} --file \"{dacpac_path}\" --account-key {storage_account_key} --auth-mode key')
    now = time.localtime()
    expiry = f'{(now.tm_year + 1)}-{now.tm_mon}-{now.tm_mday}'
    blob_sas = run_command(f'az storage blob generate-sas --container-name {storage_account_container} --name \"{dacpac_name}\" --account-name {azure_storage_account} --account-key {storage_account_key} --auth-mode key --full-uri --https-only --permissions r --expiry {expiry}', returnObject=True)

### Deploy Azure SQL Edge to the device

In [ ]:
manifest = '{\"modulesContent\":{\"$edgeAgent\":{\"properties.desired\":{\"modules\":{\"AzureSQLEdge\":{\"settings\":{\"image\":\"mcr.microsoft.com/azure-sql-edge-developer\",\"createOptions\":\"{\\\"HostConfig\\\":{\\\"CapAdd\\\":[\\\"SYS_PTRACE\\\"],\\\"Binds\\\":[\\\"sqlvolume:/sqlvolume\\\"],\\\"PortBindings\\\":{\\\"1433/tcp\\\":[{\\\"HostPort\\\":\\\"<SQL_Port>\\\"}]},\\\"Mounts\\\":[{\\\"Type\\\":\\\"volume\\\",\\\"Source\\\":\\\"sqlvolume\\\",\\\"Target\\\":\\\"/var/opt/mssql\\\"}]},\\\"User\\\":\\\"0:0\\\",\\\"Env\\\":[\\\"MSSQL_AGENT_ENABLED=TRUE\\\",\\\"ClientTransportType=AMQP_TCP_Only\\\",\\\"MSSQL_PID=Developer\\\"]}\"},\"type\":\"docker\",\"version\":\"1.0\",\"env\":{\"ACCEPT_EULA\":{\"value\":\"Y\"},\"SA_PASSWORD\":{\"value\":\"<Default_SQL_SA_Password>\"},\"MSSQL_LCID\":{\"value\":\"<SQL_LCID>\"},\"MSSQL_COLLATION\":{\"value\":\"<SQL_Collation>\"}},\"status\":\"running\",\"restartPolicy\":\"always\"}},\"runtime\":{\"settings\":{\"minDockerVersion\":\"v1.25\"},\"type\":\"docker\"},\"schemaVersion\":\"1.0\",\"systemModules\":{\"edgeAgent\":{\"settings\":{\"image\":\"mcr.microsoft.com/azureiotedge-agent:1.0\",\"createOptions\":\"\"},\"type\":\"docker\"},\"edgeHub\":{\"settings\":{\"image\":\"mcr.microsoft.com/azureiotedge-hub:1.0\",\"createOptions\":\"{\\\"HostConfig\\\":{\\\"PortBindings\\\":{\\\"443/tcp\\\":[{\\\"HostPort\\\":\\\"443\\\"}],\\\"5671/tcp\\\":[{\\\"HostPort\\\":\\\"5671\\\"}],\\\"8883/tcp\\\":[{\\\"HostPort\\\":\\\"8883\\\"}]}}}\"},\"type\":\"docker\",\"status\":\"running\",\"restartPolicy\":\"always\"}}}},\"$edgeHub\":{\"properties.desired\":{\"routes\":{},\"schemaVersion\":\"1.0\",\"storeAndForwardConfiguration\":{\"timeToLiveSecs\":7200}}},\"AzureSQLEdge\":{\"properties.desired\":{\"SqlPackage\":\"<Optional_DACPAC_ZIP_SAS_URL>\",\"ASAJobInfo\":\"<Optional_ASA_Job_ZIP_SAS_URL>\"}}}}'
manifest = manifest.replace('<Optional_DACPAC_ZIP_SAS_URL>', blob_sas).replace('<Default_SQL_SA_Password>',sa_password).replace('<SQL_LCID>',sql_lcid).replace('<SQL_Port>',sql_port).replace('<SQL_Collation>',sql_collation)
file_name = f'{uuid.uuid4().hex}.json'
manifest_file = open(file_name, 'w')
manifest_file.write(manifest)
manifest_file.close()
run_command(f'az iot edge set-modules --device-id \"{iot_device_id}\" --hub-name \"{iot_hub_name}\" --content \"{file_name}\"')
os.remove(file_name)

### **Connect to Azure SQL Edge instance in Azure Data Studio**
Click the link below to connect to the Azure SQL Edge instance, it might take a couple minutes for the service to start.

In [ ]:
from IPython.display import *
if ip_address == "":
    print('Connect to Azure SQL Edge instance feature not available because device ip address is not provided')
else:
    connectionParameter = '{"serverName":"' + f'{ip_address},{sql_port}' + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName": "sa","password":' + json.dumps(sa_password) + '}'
    display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to the Azure SQL Edge instance</font></a><br/>'))
    display(HTML('<br/><span style="color:red"><font size="2">NOTE: The Azure SQL Edge instance password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))